In [2]:
import numpy as np
import pandas as pd

In [128]:
def al(box):
    return box[:,2] - box[:,0]    
def aw(box):
    return box[:,3] - box[:,1]

def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])

def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

box_columns = ['Xmin','Ymin','Xmax','Ymax']

def weighted_averaging(data, item_id, np_data, avg):
    ix = data[data['itemId'] == item_id].index
    vi = np_data[ix]
    l = np.fmax(al(vi), np.ones(len(vi), dtype=int))
    w = np.fmax(aw(vi), np.ones(len(vi), dtype=int))
    sabx = (l*w)**2
    sb = sum(1/sabx)
    for y in range(0,4):
        avg.loc[item_id][y] = sum(vi[:,y]/sabx)/sb
        
def weighted_averaging2(vi, item_id, avg):
    l = np.fmax(al(vi), np.ones(len(vi), dtype=int))
    w = np.fmax(aw(vi), np.ones(len(vi), dtype=int))
    sabx = (l*w)**2
    sb = sum(1/sabx)
    for y in range(0,4):
        avg.loc[item_id][y] = sum(vi[:,y]/sabx)/sb         

In [114]:
train_answers = pd.read_csv("train_answers.csv")
train_votes = pd.read_csv("train_data.csv")
test_data = pd.read_csv('test_data.csv')
users_rating = pd.read_csv("users_rating.csv")
users_rating = users_rating.set_index('userId')

In [6]:
ids = list(set(test_data['itemId']))
bx = test_data.loc[: 'itemId'][box_columns]
abx = np.array(bx)

In [377]:
answer = pd.DataFrame(np.zeros((len(ids), 4), dtype=int), index=ids, columns=box_columns)
for i in ids:
    weighted_averaging(test_data, i, abx, answer)

In [151]:
iid = 33980
uids = test_data[test_data['itemId'] == iid]['userId']
uid = users_rating.loc[uids].idxmax()['iou']
ti = test_data.loc[test_data['itemId'] == iid].loc[test_data['userId'] == uid][box_columns]
# for i in box_columns:
#     answer.loc[ids[0]][i] = ti[i]

In [153]:
# print(users_rating.ix[uids])
ud = users_rating.loc[uids]
print(ud)
udx = ud['iou'].idxmax()
print(udx)
print(test_data[(test_data['itemId'] == iid) & (test_data['userId'] == udx)])

             iou
userId          
951     0.381557
410     0.439285
952     0.428113
410
      userId  itemId  Xmin  Ymin  Xmax  Ymax
1133     410   33980    60   720   161   806


In [157]:
answer = pd.DataFrame(np.zeros((len(ids), 4), dtype=int), index=ids, columns=box_columns)
for i in ids: 
    uids = test_data[test_data['itemId'] == i]['userId']
    ud = users_rating.loc[uids]
    udx = ud['iou'].idxmax()
    ti = test_data[(test_data['itemId'] == i) & (test_data['userId'] == udx)].mean()
#     ix = ti.index
#     vi = abx[ix]        
#     weighted_averaging2(vi, i, answer)
    answer.loc[i] = ti

/home/vasste/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  after removing the cwd from sys.path.


In [158]:
answer.to_csv('submit0904.csv', header=False)